In [2]:
%load_ext autoreload
%autoreload 2


In [3]:

from mmdet.apis import init_detector
import torch


In [32]:

model_init = init_detector('../configs/multimodal/mm_grounding_dino_coco.py', device='cpu')

print([k for k in model_init.state_dict().keys() if 'language' in k and 'position' in k])
saved_checkpoint = torch.load('../checkpoints/multimodal/mm_grounding_dino.pth')
model_init.load_state_dict(saved_checkpoint['state_dict'], strict = False)
print(all([torch.all(saved_checkpoint['state_dict'][k] == model_init.state_dict()[k]) for k in saved_checkpoint['state_dict'].keys() if k!='language_model.language_backbone.body.model.embeddings.position_ids']))

model_init_2 = init_detector('../configs/multimodal/mm_grounding_dino_coco.py','../checkpoints/multimodal/mm_grounding_dino.pth', device='cpu')



/localhome/aharell/CFM-Task-Models/.venv/lib/python3.10/site-packages/mmcv/cnn/bricks/transformer.py:441: DeprecationWarning: The arguments `dropout` in MultiheadAttention has been deprecated, now you can separately set `attn_drop`(float), proj_drop(float), and `dropout_layer`(dict) 
  warnings.warn(
/localhome/aharell/CFM-Task-Models/.venv/lib/python3.10/site-packages/torch/nn/modules/module.py:326: ResourceWarning: unclosed file <_io.BufferedRandom name='/tmp/tmpnc_hcvoy/tmph5gu4omw.py'>
  super().__setattr__('_is_full_backward_hook', None)
/localhome/aharell/CFM-Task-Models/.venv/lib/python3.10/site-packages/mmdet/apis/inference.py:70: UserWarning: checkpoint is None, use COCO classes by default.
  warnings.warn('checkpoint is None, use COCO classes by default.')


['language_model.language_backbone.body.model.embeddings.position_embeddings.weight']
True


/localhome/aharell/CFM-Task-Models/.venv/lib/python3.10/site-packages/mmcv/cnn/bricks/transformer.py:441: DeprecationWarning: The arguments `dropout` in MultiheadAttention has been deprecated, now you can separately set `attn_drop`(float), proj_drop(float), and `dropout_layer`(dict) 
  warnings.warn(


Loads checkpoint by local backend from path: ../checkpoints/multimodal/mm_grounding_dino.pth
The model and loaded state dict do not match exactly

unexpected key in source state_dict: language_model.language_backbone.body.model.embeddings.position_ids



In [33]:
from mmengine.runner import Runner
from mmdet.structures import DetDataSample
from typing import Union
from torch import Tensor
def to_device(data_sample:Union[dict,Tensor], device):
    if isinstance(data_sample,dict):
        for k,v in data_sample.items():
            if isinstance(v, (Tensor, DetDataSample)):
                data_sample[k] = v.to(device)
                # print(f'{k} was sent to {device}')
            elif isinstance(v, list):
                new_v = [to_device(v2, device) for v2 in v]
                data_sample[k] = new_v
                # print(f'components of {k} were sent to {device}')
            else:
                print(f'{k} could not be sent to {device}')
    elif isinstance(data_sample, (Tensor, DetDataSample)):
        data_sample = data_sample.to(device)
        # print(f'input was sent to {device}')
    elif isinstance(data_sample, list):
        data_sample = [to_device(v, device) for v in data_sample]
        # print(f'components of input were sent to {device}')
    else:
        # print(type(data_sample))
        print(f'input could not be sent to {device}')
    return data_sample

In [34]:
from mmengine.config import Config
# cfg = Config.fromfile('../configs/multimodal/mm_grounding_dino_coco.py')
# print(cfg)
cfg_tr = model_init_2.cfg.copy()
cfg_tr['val_dataloader']['dataset']['data_root'] = '../data/coco/'
cfg_tr['test_dataloader']['dataset']['data_root'] = '../data/coco/'
print(cfg_tr['test_dataloader']['dataset'])
# cfg_tr['train_dataloader']['batch_size'] = 8
# cfg_tr['train_dataloader']['num_workers'] = 1

# train_dataloader = Runner.build_dataloader(cfg_tr['train_dataloader']) 

print(cfg_tr['val_dataloader']['dataset']['ann_file'])
val_dataloader = Runner.build_dataloader(cfg_tr['val_dataloader'])

test_dataloader = Runner.build_dataloader(cfg_tr['test_dataloader'])
print(len(val_dataloader.dataset))

{'type': 'CocoDataset', 'data_root': '../data/coco/', 'ann_file': 'annotations/instances_val2017.json', 'data_prefix': {'img': 'val2017/'}, 'test_mode': True, 'pipeline': [{'type': 'LoadImageFromFile', 'backend_args': None, 'imdecode_backend': 'pillow'}, {'type': 'FixScaleResize', 'scale': (800, 1333), 'keep_ratio': True, 'backend': 'pillow'}, {'type': 'LoadAnnotations', 'with_bbox': True}, {'type': 'PackDetInputs', 'meta_keys': ('img_id', 'img_path', 'ori_shape', 'img_shape', 'scale_factor', 'text', 'custom_entities', 'tokens_positive')}], 'backend_args': None, 'return_classes': True}
annotations/instances_val2017.json
loading annotations into memory...
Done (t=0.35s)
creating index...
index created!
loading annotations into memory...
Done (t=0.19s)
creating index...
index created!
5000


In [35]:
cfg_tr['val_evaluator']['ann_file'] = cfg_tr['val_evaluator']['ann_file'].replace('data','../data') if '..' not in cfg_tr['val_evaluator']['ann_file'] else cfg_tr['val_evaluator']['ann_file']
val_eval = Runner.build_evaluator(_,cfg_tr['val_evaluator'])
print(val_eval.dataset_meta)
setattr(val_eval,'dataset_meta',val_dataloader.dataset.metainfo)
print(val_eval.dataset_meta)
# test_evaluator_cfg = deepcopy(cfg_tr['test_evaluator'])
# print(train_dataset_cfg['data_root'],cfg_tr['test_dataloader']['dataset']['ann_file'] )
# test_evaluator_cfg['ann_file'] = os.path.join(cfg_tr['test_dataloader']['dataset']['data_root'], test_evaluator_cfg['ann_file'])
# print(test_evaluator_cfg)
cfg_tr['test_evaluator']['ann_file'] = cfg_tr['test_evaluator']['ann_file'].replace('data','../data') if '..' not in cfg_tr['test_evaluator']['ann_file'] else cfg_tr['test_evaluator']['ann_file']
test_eval = Runner.build_evaluator(_,cfg_tr['test_evaluator'])
print(test_eval.dataset_meta)
setattr(test_eval,'dataset_meta',test_dataloader.dataset.metainfo)
print(test_eval.dataset_meta)
# print(val_eval.dataset_meta)

loading annotations into memory...
Done (t=0.20s)
creating index...
index created!
None
{'classes': ('person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush'), 'palette': [(220, 20, 60), (119, 11, 

In [7]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model_init_2 = model_init_2.to(device)
model_init_2.eval()
for i,data in enumerate(test_dataloader):
    with torch.no_grad():
        data = to_device(data, device)
        data = model_init_2.data_preprocessor(data)
        # print(data['inputs'].shape)
        # x = torch.tensor(data['inputs'][0], device = device)
        # samples = data['data_samples']

        out = model_init_2.predict(data['inputs'], data['data_samples'])
        test_eval.process(out)
        if (i+1)%100 == 0:
            print(f'{i+1}/{len(test_dataloader)}')

test_eval.evaluate(len(test_dataloader.dataset))

/localhome/aharell/CFM-Task-Models/.venv/lib/python3.10/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
/localhome/aharell/CFM-Task-Models/.venv/lib/python3.10/site-packages/mmcv/cnn/bricks/transformer.py:524: UserWarning: position encoding of key ismissing in MultiheadAttention.
  warnings.warn(f'position encoding of key is'


100/5000
200/5000
300/5000
400/5000
500/5000
600/5000
700/5000
800/5000
900/5000
1000/5000
1100/5000
1200/5000
1300/5000
1400/5000
1500/5000
1600/5000
1700/5000
1800/5000
1900/5000
2000/5000
2100/5000
2200/5000
2300/5000
2400/5000
2500/5000
2600/5000
2700/5000
2800/5000
2900/5000
3000/5000
3100/5000
3200/5000
3300/5000
3400/5000
3500/5000
3600/5000
3700/5000
3800/5000
3900/5000
4000/5000
4100/5000
4200/5000
4300/5000
4400/5000
4500/5000
4600/5000
4700/5000
4800/5000
4900/5000
5000/5000
07/24 14:25:07 - mmengine - INFO - Evaluating bbox...
Loading and preparing results...
DONE (t=1.77s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=44.35s).
Accumulating evaluation results...
DONE (t=19.92s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.504
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.667
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.552

{'coco/bbox_mAP': 0.504,
 'coco/bbox_mAP_50': 0.667,
 'coco/bbox_mAP_75': 0.552,
 'coco/bbox_mAP_s': 0.375,
 'coco/bbox_mAP_m': 0.533,
 'coco/bbox_mAP_l': 0.65}

In [ ]:
from cfm_task_models.split_models import SplitGDINO
device = 'cuda' if torch.cuda.is_available() else 'cpu'
split_model = SplitGDINO.create_from_cfg_and_checkpoint("../configs/multimodal/mm_grounding_dino_coco.py", '../checkpoints/multimodal/mm_grounding_dino.pth', device = device, cut_point = 2)
# split_model_2 = SplitGDINO.create_from_cfg_and_checkpoint("../configs/multimodal/mm_grounding_dino_coco.py", '../checkpoints/multimodal/mm_grounding_dino.pth', device = device, cut_point = 0)

In [ ]:
split_model = split_model.to(device)
split_model.eval()
# split_model_2 = split_model_2.to(device)
# split_model_2.eval()
model_init = model_init.to(device)
# split_model.cut_point = 1
print(split_model.data_preprocessor)
for i,data in enumerate(test_dataloader):
    with torch.no_grad():
        data = to_device(data, device)
        feats = split_model.feature_frontend(data)
        # print(feats)
        assert len(feats['inputs'])== len(data['inputs']), f'length of feats is {len(feats["inputs"])} != length of data["inputs"] which is {len(data["inputs"])}'
        out = split_model.backend_inference(feats)
        
        test_eval.process(out)
        # break
        if (i+1)%100 == 0:
            print(f'{i+1}/{len(test_dataloader)}')

test_eval.evaluate(len(test_dataloader.dataset))

In [40]:
split_model = split_model.to(device)
split_model.eval()
# split_model_2 = split_model_2.to(device)
# split_model_2.eval()
model_init = model_init.to(device)
# split_model.cut_point = 1
print(split_model.data_preprocessor)
for i,data in enumerate(val_dataloader):
    with torch.no_grad():
        data = to_device(data, device)
        feats = split_model.feature_frontend(data)
        assert len(feats['inputs'])== len(data['inputs']), f'length of feats is {len(feats["inputs"])} != length of data["inputs"] which is {len(data["inputs"])}'

        # loss = split_model.backend_loss(feats)
        out = split_model.backend_inference(feats)
    # break
        test_eval.process(out)
        if (i+1)%100 == 0:
            print(f'{i+1}/{len(test_dataloader)}')

test_eval.evaluate(len(test_dataloader.dataset))
# loss

TwoInputIdentity()
100/5000
200/5000
300/5000
400/5000
500/5000
600/5000
700/5000
800/5000
900/5000
1000/5000
1100/5000
1200/5000
1300/5000
1400/5000
1500/5000
1600/5000
1700/5000
1800/5000
1900/5000
2000/5000
2100/5000
2200/5000
2300/5000
2400/5000
2500/5000
2600/5000
2700/5000
2800/5000
2900/5000
3000/5000
3100/5000
3200/5000
3300/5000
3400/5000
3500/5000
3600/5000
3700/5000
3800/5000
3900/5000
4000/5000
4100/5000
4200/5000
4300/5000
4400/5000
4500/5000
4600/5000
4700/5000
4800/5000
4900/5000
5000/5000
07/26 15:20:52 - mmengine - INFO - Evaluating bbox...
Loading and preparing results...
DONE (t=1.91s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=45.20s).
Accumulating evaluation results...
DONE (t=19.56s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.504
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.667
 Average Precision  (AP) @[ IoU=0.75      | area=   all | max

{'coco/bbox_mAP': 0.504,
 'coco/bbox_mAP_50': 0.667,
 'coco/bbox_mAP_75': 0.552,
 'coco/bbox_mAP_s': 0.375,
 'coco/bbox_mAP_m': 0.533,
 'coco/bbox_mAP_l': 0.65}

In [ ]:
!wget 'https://download.openmmlab.com/mmdetection/v3.0/glip/glip_tiny_mmdet-c24ce662.pth' -O ../checkpoints/multimodal/glip.pth

--2024-07-17 16:02:53--  https://download.openmmlab.com/mmdetection/v3.0/glip/glip_tiny_mmdet-c24ce662.pth
Resolving download.openmmlab.com (download.openmmlab.com)... 8.25.82.176, 8.25.82.179, 8.25.82.177, ...
Connecting to download.openmmlab.com (download.openmmlab.com)|8.25.82.176|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 927536362 (885M) [application/octet-stream]
Saving to: ‘../checkpoints/multimodal/glip.pth’

                ../   3%[                    ]  29.43M   492KB/s    eta 4h 30m ^C


In [ ]:
glip_model = init_detector('../configs/multimodal/glip_t.py','../checkpoints/multimodal/glip.pth', device='cpu')

Loads checkpoint by local backend from path: ../checkpoints/multimodal/glip.pth
The model and loaded state dict do not match exactly

unexpected key in source state_dict: language_model.language_backbone.body.model.embeddings.position_ids



In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
glip_model = glip_model.to(device)
glip_model.eval()
for i,data in enumerate(test_dataloader):
    with torch.no_grad():
        data = to_device(data, device)
        data = glip_model.data_preprocessor(data)
        # print(data['inputs'].shape)
        # x = torch.tensor(data['inputs'][0], device = device)
        # samples = data['data_samples']

        out = glip_model.predict(data['inputs'], data['data_samples'])
        test_eval.process(out)
        if (i+1)%100 == 0:
            print(f'{i+1}/{len(test_dataloader)}')

test_eval.evaluate(len(test_dataloader.dataset))

100/5000
200/5000
300/5000
400/5000
500/5000
600/5000
700/5000
800/5000
900/5000
1000/5000
1100/5000
1200/5000
1300/5000
1400/5000
1500/5000
1600/5000
1700/5000
1800/5000
1900/5000
2000/5000
2100/5000
2200/5000
2300/5000
2400/5000
2500/5000
2600/5000
2700/5000
2800/5000
2900/5000
3000/5000
3100/5000
3200/5000
3300/5000
3400/5000
3500/5000
3600/5000
3700/5000
3800/5000
3900/5000
4000/5000
4100/5000
4200/5000
4300/5000
4400/5000
4500/5000
4600/5000
4700/5000
4800/5000
4900/5000
5000/5000
07/18 11:55:25 - mmengine - INFO - Evaluating bbox...
Loading and preparing results...
DONE (t=0.38s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=17.87s).
Accumulating evaluation results...
DONE (t=4.67s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.552
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.735
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.606


{'coco/bbox_mAP': 0.552,
 'coco/bbox_mAP_50': 0.735,
 'coco/bbox_mAP_75': 0.606,
 'coco/bbox_mAP_s': 0.409,
 'coco/bbox_mAP_m': 0.599,
 'coco/bbox_mAP_l': 0.691}

In [ ]:
from cfm_task_models.split_models import SplitGLIP
glip_split = SplitGLIP.create_from_cfg_and_checkpoint("../configs/multimodal/glip_t.py", '../checkpoints/multimodal/glip.pth', device = device, cut_point = 1)

Loads checkpoint by local backend from path: ../checkpoints/multimodal/glip.pth
The model and loaded state dict do not match exactly

unexpected key in source state_dict: language_model.language_backbone.body.model.embeddings.position_ids



In [ ]:

glip_split = glip_split.to(device)
glip_split.eval()
# split_model_2 = split_model_2.to(device)
# split_model_2.eval()
# split_model.cut_point = 1
print(glip_split.data_preprocessor)
for i,data in enumerate(val_dataloader):
    # with torch.no_grad():
    data = to_device(data, device)
    feats = glip_split.feature_frontend(data)
    # assert len(feats['inputs'])== len(data['inputs']), f'length of feats is {len(feats["inputs"])} != length of data["inputs"] which is {len(data["inputs"])}'
    
    # loss = split_model.backend_loss(feats)
    out = glip_split.backend_inference(feats)
    # break
    test_eval.process(out)
    if (i+1)%100 == 0:
        print(f'{i+1}/{len(test_dataloader)}')

test_eval.evaluate(len(test_dataloader.dataset))

TwoInputIdentity()


/localhome/aharell/CFM-Task-Models/.venv/lib/python3.10/site-packages/transformers/modeling_utils.py:1060: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
/localhome/aharell/CFM-Task-Models/.venv/lib/python3.10/site-packages/torch/nn/functional.py:4070: UserWarning: nn.functional.upsample_bilinear is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample_bilinear is deprecated. Use nn.functional.interpolate instead.")


100/5000
200/5000
300/5000
400/5000
500/5000
600/5000
700/5000
800/5000
900/5000
1000/5000
1100/5000
1200/5000
1300/5000
1400/5000
1500/5000
1600/5000
1700/5000
1800/5000
1900/5000
2000/5000
2100/5000
2200/5000
2300/5000
2400/5000
2500/5000
2600/5000
2700/5000
2800/5000
2900/5000
3000/5000
3100/5000
3200/5000
3300/5000
3400/5000
3500/5000
3600/5000
3700/5000
3800/5000
3900/5000
4000/5000
4100/5000
4200/5000
4300/5000
4400/5000
4500/5000
4600/5000
4700/5000
4800/5000
4900/5000
5000/5000
07/18 12:21:36 - mmengine - INFO - Evaluating bbox...
Loading and preparing results...
DONE (t=0.58s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=18.21s).
Accumulating evaluation results...
DONE (t=4.73s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.552
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.735
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.606


{'coco/bbox_mAP': 0.552,
 'coco/bbox_mAP_50': 0.735,
 'coco/bbox_mAP_75': 0.606,
 'coco/bbox_mAP_s': 0.409,
 'coco/bbox_mAP_m': 0.599,
 'coco/bbox_mAP_l': 0.691}

In [11]:
from mmdet.models import Mask2Former
%load_ext autoreload
%autoreload 2

from mmdet.apis import init_detector
import torch
mask2former_swin = init_detector('../configs/detection/mask2former_swin_t.py','../checkpoints/detection/mask2former_swin_instance.pth', device='cpu')

# Mask2former needs separate dataloaders and evaluators because it needs masks etc.

cfg_tr = mask2former_swin.cfg.copy()
cfg_tr['val_dataloader']['dataset']['data_root'] = '../data/coco/'
cfg_tr['test_dataloader']['dataset']['data_root'] = '../data/coco/'
print(cfg_tr['test_dataloader']['dataset'])
# cfg_tr['train_dataloader']['batch_size'] = 8
# cfg_tr['train_dataloader']['num_workers'] = 1

# train_dataloader = Runner.build_dataloader(cfg_tr['train_dataloader']) 

print(cfg_tr['val_dataloader']['dataset']['ann_file'])
val_dataloader = Runner.build_dataloader(cfg_tr['val_dataloader'])

test_dataloader = Runner.build_dataloader(cfg_tr['test_dataloader'])
print(len(val_dataloader.dataset))

cfg_tr['val_evaluator']['ann_file'] = cfg_tr['val_evaluator']['ann_file'].replace('data','../data') if '..' not in cfg_tr['val_evaluator']['ann_file'] else cfg_tr['val_evaluator']['ann_file']
val_eval = Runner.build_evaluator(_,cfg_tr['val_evaluator'])
print(val_eval.dataset_meta)
setattr(val_eval,'dataset_meta',val_dataloader.dataset.metainfo)
print(val_eval.dataset_meta)
# test_evaluator_cfg = deepcopy(cfg_tr['test_evaluator'])
# print(train_dataset_cfg['data_root'],cfg_tr['test_dataloader']['dataset']['ann_file'] )
# test_evaluator_cfg['ann_file'] = os.path.join(cfg_tr['test_dataloader']['dataset']['data_root'], test_evaluator_cfg['ann_file'])
# print(test_evaluator_cfg)
cfg_tr['test_evaluator']['ann_file'] = cfg_tr['test_evaluator']['ann_file'].replace('data','../data') if '..' not in cfg_tr['test_evaluator']['ann_file'] else cfg_tr['test_evaluator']['ann_file']
test_eval = Runner.build_evaluator(_,cfg_tr['test_evaluator'])
print(test_eval.dataset_meta)
setattr(test_eval,'dataset_meta',test_dataloader.dataset.metainfo)
print(test_eval.dataset_meta)
# print(val_eval.dataset_meta)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Loads checkpoint by local backend from path: ../checkpoints/detection/mask2former_swin_instance.pth
{'type': 'CocoDataset', 'data_root': '../data/coco/', 'ann_file': 'annotations/instances_val2017.json', 'data_prefix': {'img': 'val2017/', 'seg': 'annotations/panoptic_val2017/'}, 'test_mode': True, 'pipeline': [{'type': 'LoadImageFromFile', 'to_float32': True, 'backend_args': None}, {'type': 'Resize', 'scale': (1333, 800), 'keep_ratio': True}, {'type': 'LoadAnnotations', 'with_bbox': True, 'with_mask': True}, {'type': 'PackDetInputs', 'meta_keys': ('img_id', 'img_path', 'ori_shape', 'img_shape', 'scale_factor')}], 'backend_args': None}
annotations/instances_val2017.json
loading annotations into memory...
Done (t=0.20s)
creating index...
index created!
loading annotations into memory...
Done (t=0.19s)
creating index...
index created!
5000
loading annotations into memory...
Done (t=0.19s)
creating inde

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
mask2former_swin = mask2former_swin.to(device)
mask2former_swin.eval()
for i,data in enumerate(test_dataloader):
    with torch.no_grad():
        data = to_device(data, device)
        data = mask2former_swin.data_preprocessor(data)
        # print(data['inputs'].shape)
        # x = torch.tensor(data['inputs'][0], device = device)
        # samples = data['data_samples']

        out = mask2former_swin.predict(data['inputs'], data['data_samples'])
        test_eval.process(out)
        if (i+1)%100 == 0:
            print(f'{i+1}/{len(test_dataloader)}')

test_eval.evaluate(len(test_dataloader.dataset))

100/5000
200/5000
300/5000
400/5000
500/5000
600/5000
700/5000
800/5000
900/5000
1000/5000
1100/5000
1200/5000
1300/5000
1400/5000
1500/5000
1600/5000
1700/5000
1800/5000
1900/5000
2000/5000
2100/5000
2200/5000
2300/5000
2400/5000
2500/5000
2600/5000
2700/5000
2800/5000
2900/5000
3000/5000
3100/5000
3200/5000
3300/5000
3400/5000
3500/5000
3600/5000
3700/5000
3800/5000
3900/5000
4000/5000
4100/5000
4200/5000
4300/5000
4400/5000
4500/5000
4600/5000
4700/5000
4800/5000
4900/5000
5000/5000
07/19 15:35:21 - mmengine - INFO - Evaluating bbox...
Loading and preparing results...
DONE (t=0.74s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=21.02s).
Accumulating evaluation results...
DONE (t=6.92s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.477
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.676
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.511


{'coco/bbox_mAP': 0.477,
 'coco/bbox_mAP_50': 0.676,
 'coco/bbox_mAP_75': 0.511,
 'coco/bbox_mAP_s': 0.294,
 'coco/bbox_mAP_m': 0.506,
 'coco/bbox_mAP_l': 0.654,
 'coco/segm_mAP': 0.447,
 'coco/segm_mAP_50': 0.679,
 'coco/segm_mAP_75': 0.481,
 'coco/segm_mAP_s': 0.241,
 'coco/segm_mAP_m': 0.481,
 'coco/segm_mAP_l': 0.67}

In [ ]:
from cfm_task_models.split_models import SplitMask2Former
device = 'cuda' if torch.cuda.is_available() else 'cpu'
mask2former_swin_split = SplitMask2Former.create_from_cfg_and_checkpoint("../configs/detection/mask2former_swin_t.py", '../checkpoints/detection/mask2former_instance.pth', device = device, cut_point = 1)

Loads checkpoint by local backend from path: ../checkpoints/detection/mask2former_instance.pth


In [ ]:
print(mask2former_swin_split.frontend_preprocessor)
print(mask2former_swin.data_preprocessor)


DetDataPreprocessor(
  (batch_augments): ModuleList(
    (0): BatchFixedSizePad()
  )
)
DetDataPreprocessor(
  (batch_augments): ModuleList(
    (0): BatchFixedSizePad()
  )
)


False

In [ ]:
from cfm_task_models.split_utils import SplitSwinFeatures
mask2former_swin_split = mask2former_swin_split.to(device)
mask2former_swin_split.eval()
# split_model_2 = split_model_2.to(device)
# split_model_2.eval()
# split_model.cut_point = 1
print(mask2former_swin_split.frontend_preprocessor)
for i,data in enumerate(test_dataloader):
    with torch.no_grad():
    # break
    # data = to_device(data, device)
    # print(data)
    # data = mask2former_swin.data_preprocessor(data)
    # print(data)
        # feats = mask2former_swin_split.frontend_preprocessor(data)
        # print(feats)
        # out =  mask2former_swin_split.backbone.split_forward_v2(feats['inputs'], output_layer = mask2former_swin_split.cut_point-1)
        #     # data['inputs'] = {"hw_shape": out[0],
        #     #                   "outs": out[1]}
        # feats['inputs'] = SplitSwinFeatures(outs=out[1], hw_shape=out[0], device=device)
        feats = mask2former_swin_split.feature_frontend(data)
        # assert len(feats['inputs'])== len(data['inputs']), f'length of feats is {len(feats["inputs"])} != length of data["inputs"] which is {len(data["inputs"])}'
        
        # loss = split_model.backend_loss(feats)
        out = mask2former_swin_split.backend_inference(feats)
        # break
        test_eval.process(out)
        # break
        if (i+1)%100 == 0:
            print(f'{i+1}/{len(test_dataloader)}')

test_eval.evaluate(len(test_dataloader.dataset))

DetDataPreprocessor(
  (batch_augments): ModuleList(
    (0): BatchFixedSizePad()
  )
)
100/5000
200/5000
300/5000
400/5000
500/5000
600/5000
700/5000
800/5000
900/5000
1000/5000
1100/5000
1200/5000
1300/5000
1400/5000
1500/5000
1600/5000
1700/5000
1800/5000
1900/5000
2000/5000
2100/5000
2200/5000
2300/5000
2400/5000
2500/5000
2600/5000
2700/5000
2800/5000
2900/5000
3000/5000
3100/5000
3200/5000
3300/5000
3400/5000
3500/5000
3600/5000
3700/5000
3800/5000
3900/5000
4000/5000
4100/5000
4200/5000
4300/5000
4400/5000
4500/5000
4600/5000
4700/5000
4800/5000
4900/5000
5000/5000
07/24 09:57:19 - mmengine - INFO - Evaluating bbox...
Loading and preparing results...
DONE (t=0.69s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=20.81s).
Accumulating evaluation results...
DONE (t=6.99s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.477
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] =

{'coco/bbox_mAP': 0.477,
 'coco/bbox_mAP_50': 0.676,
 'coco/bbox_mAP_75': 0.511,
 'coco/bbox_mAP_s': 0.294,
 'coco/bbox_mAP_m': 0.506,
 'coco/bbox_mAP_l': 0.654,
 'coco/segm_mAP': 0.447,
 'coco/segm_mAP_50': 0.679,
 'coco/segm_mAP_75': 0.481,
 'coco/segm_mAP_s': 0.241,
 'coco/segm_mAP_m': 0.481,
 'coco/segm_mAP_l': 0.67}

In [5]:
from mmdet.models import Mask2Former
%load_ext autoreload
%autoreload 2

from mmdet.apis import init_detector
import torch
mask2former_r50 = init_detector('../configs/detection/mask2former_r50.py','../checkpoints/detection/mask2former_r50.pth', device='cpu')

# Mask2former needs separate dataloaders and evaluators because it needs masks etc.

cfg_tr = mask2former_r50.cfg.copy()
cfg_tr['val_dataloader']['dataset']['data_root'] = '../data/coco/'
cfg_tr['test_dataloader']['dataset']['data_root'] = '../data/coco/'
print(cfg_tr['test_dataloader']['dataset'])
# cfg_tr['train_dataloader']['batch_size'] = 8
# cfg_tr['train_dataloader']['num_workers'] = 1

# train_dataloader = Runner.build_dataloader(cfg_tr['train_dataloader']) 

print(cfg_tr['val_dataloader']['dataset']['ann_file'])
val_dataloader = Runner.build_dataloader(cfg_tr['val_dataloader'])

test_dataloader = Runner.build_dataloader(cfg_tr['test_dataloader'])
print(len(val_dataloader.dataset))

cfg_tr['val_evaluator']['ann_file'] = cfg_tr['val_evaluator']['ann_file'].replace('data','../data') if '..' not in cfg_tr['val_evaluator']['ann_file'] else cfg_tr['val_evaluator']['ann_file']
val_eval = Runner.build_evaluator(_,cfg_tr['val_evaluator'])
print(val_eval.dataset_meta)
setattr(val_eval,'dataset_meta',val_dataloader.dataset.metainfo)
print(val_eval.dataset_meta)
# test_evaluator_cfg = deepcopy(cfg_tr['test_evaluator'])
# print(train_dataset_cfg['data_root'],cfg_tr['test_dataloader']['dataset']['ann_file'] )
# test_evaluator_cfg['ann_file'] = os.path.join(cfg_tr['test_dataloader']['dataset']['data_root'], test_evaluator_cfg['ann_file'])
# print(test_evaluator_cfg)
cfg_tr['test_evaluator']['ann_file'] = cfg_tr['test_evaluator']['ann_file'].replace('data','../data') if '..' not in cfg_tr['test_evaluator']['ann_file'] else cfg_tr['test_evaluator']['ann_file']
test_eval = Runner.build_evaluator(_,cfg_tr['test_evaluator'])
print(test_eval.dataset_meta)
setattr(test_eval,'dataset_meta',test_dataloader.dataset.metainfo)
print(test_eval.dataset_meta)
# print(val_eval.dataset_meta)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Loads checkpoint by local backend from path: ../checkpoints/detection/mask2former_r50.pth


{'type': 'CocoDataset', 'data_root': '../data/coco/', 'ann_file': 'annotations/instances_val2017.json', 'data_prefix': {'img': 'val2017/', 'seg': 'annotations/panoptic_val2017/'}, 'test_mode': True, 'pipeline': [{'type': 'LoadImageFromFile', 'to_float32': True, 'backend_args': None}, {'type': 'Resize', 'scale': (1333, 800), 'keep_ratio': True}, {'type': 'LoadAnnotations', 'with_bbox': True, 'with_mask': True}, {'type': 'PackDetInputs', 'meta_keys': ('img_id', 'img_path', 'ori_shape', 'img_shape', 'scale_factor')}], 'backend_args': None}
annotations/instances_val2017.json
loading annotations into memory...
Done (t=0.27s)
creating index...
index created!
loading annotations into memory...
Done (t=0.27s)
creating index...
index created!
5000
loading annotations into memory...
Done (t=0.26s)
creating index...
index created!
None
{'classes': ('person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 

In [25]:
from cfm_task_models.split_models import SplitMask2Former
device = 'cuda' if torch.cuda.is_available() else 'cpu'
mask2former_r50_split = SplitMask2Former.create_from_cfg_and_checkpoint("../configs/detection/mask2former_r50.py", '../checkpoints/detection/mask2former_r50.pth', device = device, cut_point = 2)

Loads checkpoint by local backend from path: ../checkpoints/detection/mask2former_r50.pth


In [26]:
mask2former_r50_split = mask2former_r50_split.to(device)
mask2former_r50_split.eval()
# split_model_2 = split_model_2.to(device)
# split_model_2.eval()
# split_model.cut_point = 1
print(mask2former_r50_split.frontend_preprocessor)
for i,data in enumerate(test_dataloader):
    with torch.no_grad():
    # break
    # data = to_device(data, device)
    # print(data)
    # data = mask2former_swin.data_preprocessor(data)
    # print(data)
        # feats = mask2former_swin_split.frontend_preprocessor(data)
        # print(feats)
        # out =  mask2former_swin_split.backbone.split_forward_v2(feats['inputs'], output_layer = mask2former_swin_split.cut_point-1)
        #     # data['inputs'] = {"hw_shape": out[0],
        #     #                   "outs": out[1]}
        # feats['inputs'] = SplitSwinFeatures(outs=out[1], hw_shape=out[0], device=device)
        feats = mask2former_r50_split.feature_frontend(data)
        # assert len(feats['inputs'])== len(data['inputs']), f'length of feats is {len(feats["inputs"])} != length of data["inputs"] which is {len(data["inputs"])}'
        # print([t.shape for t in feats['inputs'].outs])
        
        # break
        # loss = split_model.backend_loss(feats)
        # print(len(feats['inputs'].outs))
        out = mask2former_r50_split.backend_inference(feats)
        # break
        test_eval.process(out)
        # break
        if (i+1)%100 == 0:
            print(f'{i+1}/{len(test_dataloader)}')

test_eval.evaluate(len(test_dataloader.dataset))

DetDataPreprocessor(
  (batch_augments): ModuleList(
    (0): BatchFixedSizePad()
  )
)
100/5000
200/5000
300/5000
400/5000
500/5000
600/5000
700/5000
800/5000
900/5000
1000/5000
1100/5000
1200/5000
1300/5000
1400/5000
1500/5000
1600/5000
1700/5000
1800/5000
1900/5000
2000/5000
2100/5000
2200/5000
2300/5000
2400/5000
2500/5000
2600/5000
2700/5000
2800/5000
2900/5000
3000/5000
3100/5000
3200/5000
3300/5000
3400/5000
3500/5000
3600/5000
3700/5000
3800/5000
3900/5000
4000/5000
4100/5000
4200/5000
4300/5000
4400/5000
4500/5000
4600/5000
4700/5000
4800/5000
4900/5000
5000/5000
07/26 14:47:37 - mmengine - INFO - Evaluating bbox...
Loading and preparing results...
DONE (t=1.08s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=20.75s).
Accumulating evaluation results...
DONE (t=6.85s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.457
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] =

{'coco/bbox_mAP': 0.457,
 'coco/bbox_mAP_50': 0.651,
 'coco/bbox_mAP_75': 0.491,
 'coco/bbox_mAP_s': 0.268,
 'coco/bbox_mAP_m': 0.485,
 'coco/bbox_mAP_l': 0.626,
 'coco/segm_mAP': 0.429,
 'coco/segm_mAP_50': 0.652,
 'coco/segm_mAP_75': 0.46,
 'coco/segm_mAP_s': 0.22,
 'coco/segm_mAP_m': 0.464,
 'coco/segm_mAP_l': 0.647}

In [54]:
from cfm_task_models.split_utils import NetworkSplitter
from mmdet.models import RetinaNet
SplitRetinaNet = NetworkSplitter()(RetinaNet)
split_retina = SplitRetinaNet.create_from_cfg_and_checkpoint("../configs/detection/retinanet_r50.py", '../checkpoints/detection/retinanet_r50.pth', device = device, cut_point = 2)

Loads checkpoint by local backend from path: ../checkpoints/detection/retinanet_r50.pth


In [7]:
retinanet = init_detector('../configs/detection/retinanet_r50.py','../checkpoints/detection/retinanet_r50.pth', device='cpu')
cfg_tr = retinanet.cfg.copy()
cfg_tr['val_dataloader']['dataset']['data_root'] = '../data/coco/'
cfg_tr['test_dataloader']['dataset']['data_root'] = '../data/coco/'
print(cfg_tr['test_dataloader']['dataset'])
# cfg_tr['train_dataloader']['batch_size'] = 8
# cfg_tr['train_dataloader']['num_workers'] = 1

# train_dataloader = Runner.build_dataloader(cfg_tr['train_dataloader']) 

print(cfg_tr['val_dataloader']['dataset']['ann_file'])
val_dataloader = Runner.build_dataloader(cfg_tr['val_dataloader'])

test_dataloader = Runner.build_dataloader(cfg_tr['test_dataloader'])
print(len(val_dataloader.dataset))

cfg_tr['val_evaluator']['ann_file'] = cfg_tr['val_evaluator']['ann_file'].replace('data','../data') if '..' not in cfg_tr['val_evaluator']['ann_file'] else cfg_tr['val_evaluator']['ann_file']
val_eval = Runner.build_evaluator(_,cfg_tr['val_evaluator'])
print(val_eval.dataset_meta)
setattr(val_eval,'dataset_meta',val_dataloader.dataset.metainfo)
print(val_eval.dataset_meta)
# test_evaluator_cfg = deepcopy(cfg_tr['test_evaluator'])
# print(train_dataset_cfg['data_root'],cfg_tr['test_dataloader']['dataset']['ann_file'] )
# test_evaluator_cfg['ann_file'] = os.path.join(cfg_tr['test_dataloader']['dataset']['data_root'], test_evaluator_cfg['ann_file'])
# print(test_evaluator_cfg)
cfg_tr['test_evaluator']['ann_file'] = cfg_tr['test_evaluator']['ann_file'].replace('data','../data') if '..' not in cfg_tr['test_evaluator']['ann_file'] else cfg_tr['test_evaluator']['ann_file']
test_eval = Runner.build_evaluator(_,cfg_tr['test_evaluator'])
print(test_eval.dataset_meta)
setattr(test_eval,'dataset_meta',test_dataloader.dataset.metainfo)
print(test_eval.dataset_meta)
# print(val_eval.dataset_meta)


Loads checkpoint by local backend from path: ../checkpoints/detection/retinanet_r50.pth
{'type': 'CocoDataset', 'data_root': '../data/coco/', 'ann_file': 'annotations/instances_val2017.json', 'data_prefix': {'img': 'val2017/'}, 'test_mode': True, 'pipeline': [{'type': 'LoadImageFromFile', 'backend_args': None}, {'type': 'Resize', 'scale': (1333, 800), 'keep_ratio': True}, {'type': 'LoadAnnotations', 'with_bbox': True}, {'type': 'PackDetInputs', 'meta_keys': ('img_id', 'img_path', 'ori_shape', 'img_shape', 'scale_factor')}], 'backend_args': None}
annotations/instances_val2017.json
loading annotations into memory...
Done (t=0.28s)
creating index...
index created!
loading annotations into memory...
Done (t=0.28s)
creating index...
index created!
5000
loading annotations into memory...
Done (t=0.28s)
creating index...
index created!
None
{'classes': ('person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking

In [59]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
retinanet = retinanet.to(device)
retinanet.eval()
for i,data in enumerate(test_dataloader):
    with torch.no_grad():
        data = to_device(data, device)
        data = retinanet.data_preprocessor(data)
        # print(data['inputs'].shape)
        # x = torch.tensor(data['inputs'][0], device = device)
        # samples = data['data_samples']

        out = retinanet.predict(data['inputs'], data['data_samples'])
        test_eval.process(out)
        if (i+1)%100 == 0:
            print(f'{i+1}/{len(test_dataloader)}')

test_eval.evaluate(len(test_dataloader.dataset))

100/5000
200/5000
300/5000
400/5000
500/5000
600/5000
700/5000
800/5000
900/5000
1000/5000
1100/5000
1200/5000
1300/5000
1400/5000
1500/5000
1600/5000
1700/5000
1800/5000
1900/5000
2000/5000
2100/5000
2200/5000
2300/5000
2400/5000
2500/5000
2600/5000
2700/5000
2800/5000
2900/5000
3000/5000
3100/5000
3200/5000
3300/5000
3400/5000
3500/5000
3600/5000
3700/5000
3800/5000
3900/5000
4000/5000
4100/5000
4200/5000
4300/5000
4400/5000
4500/5000
4600/5000
4700/5000
4800/5000
4900/5000
5000/5000
07/26 16:24:43 - mmengine - INFO - Evaluating bbox...
Loading and preparing results...
DONE (t=0.80s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=22.62s).
Accumulating evaluation results...
DONE (t=6.36s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.390
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.594
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.417


{'coco/bbox_mAP': 0.39,
 'coco/bbox_mAP_50': 0.594,
 'coco/bbox_mAP_75': 0.417,
 'coco/bbox_mAP_s': 0.234,
 'coco/bbox_mAP_m': 0.427,
 'coco/bbox_mAP_l': 0.522}

In [60]:
split_retina = split_retina.to(device)
split_retina.eval()
# split_model_2 = split_model_2.to(device)
# split_model_2.eval()
# split_model.cut_point = 1
print(split_retina.frontend_preprocessor)
for i,data in enumerate(test_dataloader):
    with torch.no_grad():
    # break
    # data = to_device(data, device)
    # print(data)
    # data = mask2former_swin.data_preprocessor(data)
    # print(data)
        # feats = mask2former_swin_split.frontend_preprocessor(data)
        # print(feats)
        # out =  mask2former_swin_split.backbone.split_forward_v2(feats['inputs'], output_layer = mask2former_swin_split.cut_point-1)
        #     # data['inputs'] = {"hw_shape": out[0],
        #     #                   "outs": out[1]}
        # feats['inputs'] = SplitSwinFeatures(outs=out[1], hw_shape=out[0], device=device)
        feats = split_retina.feature_frontend(data)
        # assert len(feats['inputs'])== len(data['inputs']), f'length of feats is {len(feats["inputs"])} != length of data["inputs"] which is {len(data["inputs"])}'
        # print([t.shape for t in feats['inputs'].outs])
        
        # break
        # loss = split_model.backend_loss(feats)
        # print(len(feats['inputs'].outs))
        out = split_retina.backend_inference(feats)
        # break
        test_eval.process(out)
        # break
        if (i+1)%100 == 0:
            print(f'{i+1}/{len(test_dataloader)}')

test_eval.evaluate(len(test_dataloader.dataset))

DetDataPreprocessor()
100/5000
200/5000
300/5000
400/5000
500/5000
600/5000
700/5000
800/5000
900/5000
1000/5000
1100/5000
1200/5000
1300/5000
1400/5000
1500/5000
1600/5000
1700/5000
1800/5000
1900/5000
2000/5000
2100/5000
2200/5000
2300/5000
2400/5000
2500/5000
2600/5000
2700/5000
2800/5000
2900/5000
3000/5000
3100/5000
3200/5000
3300/5000
3400/5000
3500/5000
3600/5000
3700/5000
3800/5000
3900/5000
4000/5000
4100/5000
4200/5000
4300/5000
4400/5000
4500/5000
4600/5000
4700/5000
4800/5000
4900/5000
5000/5000
07/26 16:28:14 - mmengine - INFO - Evaluating bbox...
Loading and preparing results...
DONE (t=0.55s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=21.86s).
Accumulating evaluation results...
DONE (t=6.79s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.390
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.594
 Average Precision  (AP) @[ IoU=0.75      | area=   all | m

{'coco/bbox_mAP': 0.39,
 'coco/bbox_mAP_50': 0.594,
 'coco/bbox_mAP_75': 0.417,
 'coco/bbox_mAP_s': 0.234,
 'coco/bbox_mAP_m': 0.427,
 'coco/bbox_mAP_l': 0.522}

In [16]:
from cfm_task_models.split_utils import NetworkSplitter
from mmdet.models import RetinaNet

@NetworkSplitter() # checking the use of Networksplitter as a decorator + adding extra init stuff
class SplitRetinaNet(RetinaNet):
    def __init__(self, *args, **kwargs):
        self.check_value = True

device = 'cuda' if torch.cuda.is_available() else 'cpu'

split_retina_2 = SplitRetinaNet.create_from_cfg_and_checkpoint("../configs/detection/retinanet_r50.py", '../checkpoints/detection/retinanet_r50.pth', device = device, cut_point = 2)
print(split_retina_2.check_value)

Loads checkpoint by local backend from path: ../checkpoints/detection/retinanet_r50.pth
True


In [18]:
'feature_frontend' in dir(split_retina_2)

True

In [13]:
split_retina_2 = split_retina_2.to(device)
split_retina_2.eval()
# split_model_2 = split_model_2.to(device)
# split_model_2.eval()
# split_model.cut_point = 1
print(split_retina_2.frontend_preprocessor)
for i,data in enumerate(test_dataloader):
    with torch.no_grad():
    # break
    # data = to_device(data, device)
    # print(data)
    # data = mask2former_swin.data_preprocessor(data)
    # print(data)
        # feats = mask2former_swin_split.frontend_preprocessor(data)
        # print(feats)
        # out =  mask2former_swin_split.backbone.split_forward_v2(feats['inputs'], output_layer = mask2former_swin_split.cut_point-1)
        #     # data['inputs'] = {"hw_shape": out[0],
        #     #                   "outs": out[1]}
        # feats['inputs'] = SplitSwinFeatures(outs=out[1], hw_shape=out[0], device=device)
        feats = split_retina_2.feature_frontend(data)
        # assert len(feats['inputs'])== len(data['inputs']), f'length of feats is {len(feats["inputs"])} != length of data["inputs"] which is {len(data["inputs"])}'
        # print([t.shape for t in feats['inputs'].outs])
        
        # break
        # loss = split_model.backend_loss(feats)
        # print(len(feats['inputs'].outs))
        out = split_retina_2.backend_inference(feats)
        # break
        test_eval.process(out)
        # break
        if (i+1)%100 == 0:
            print(f'{i+1}/{len(test_dataloader)}')

test_eval.evaluate(len(test_dataloader.dataset))

DetDataPreprocessor()
100/5000
200/5000
300/5000
400/5000
500/5000
600/5000
700/5000
800/5000
900/5000
1000/5000
1100/5000
1200/5000
1300/5000
1400/5000
1500/5000
1600/5000
1700/5000
1800/5000
1900/5000
2000/5000
2100/5000
2200/5000
2300/5000
2400/5000
2500/5000
2600/5000
2700/5000
2800/5000
2900/5000
3000/5000
3100/5000
3200/5000
3300/5000
3400/5000
3500/5000
3600/5000
3700/5000
3800/5000
3900/5000
4000/5000
4100/5000
4200/5000
4300/5000
4400/5000
4500/5000
4600/5000
4700/5000
4800/5000
4900/5000
5000/5000
08/02 14:22:28 - mmengine - INFO - Evaluating bbox...
Loading and preparing results...
DONE (t=0.64s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=21.04s).
Accumulating evaluation results...
DONE (t=6.16s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.390
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.594
 Average Precision  (AP) @[ IoU=0.75      | area=   all | m

{'coco/bbox_mAP': 0.39,
 'coco/bbox_mAP_50': 0.594,
 'coco/bbox_mAP_75': 0.417,
 'coco/bbox_mAP_s': 0.234,
 'coco/bbox_mAP_m': 0.427,
 'coco/bbox_mAP_l': 0.522}

In [26]:
from cfm_task_models.legacy import RepPointsV2MaskDetector, convert_swin_checkpoint_file
from cfm_task_models.split_utils import NetworkSplitter
from cfm_task_models.split_models import SplitRepPointsV2MaskDetector
from mmengine.config import Config

@NetworkSplitter()
class AutoSplitRepPointsV2MaskDetector(RepPointsV2MaskDetector):
    def __init__(self, *args, **kwargs):
        pass
    @classmethod
    def create_from_cfg_and_checkpoint(cls, cfg_path, checkpoint_path, device = 'cpu', cut_point=1):
        cfg = Config.fromfile(cfg_path)
        # print(cfg.model)
        if not 'converted' in checkpoint_path:
            dst = checkpoint_path.replace('.pth', '_converted.pth')
            checkpoint_path = convert_swin_checkpoint_file(checkpoint_path, dst)
        
        model_init = init_detector(cfg_path, checkpoint_path, device=device)
        model = cls.create_from_instance_and_cfg(model_init, cfg, cut_point = cut_point)
        # model.prepare_preprocessing()
        return model

    def backend_inference(self, data):
        result = self.val_step(data)
        return result #data['data_samples']
    
    def backend_loss(self, data):
        if isinstance(data['data_samples'][0], DetDataSample):
            img_metas = []
            gt_bboxes = []
            gt_labels = []
            gt_bboxes_ignore = []
            gt_sem_map= []
            gt_sem_weights = []
            gt_masks = []
            for ds in data['data_samples']:
                img_metas.append(ds.metainfo)
                gt_bboxes.append(ds.gt_instances.bboxes)
                gt_labels.append(ds.gt_instances.labels)
                gt_bboxes_ignore.append(ds.ignored_instances.bboxes)
                gt_masks.append(ds.gt_instances.masks)
                gt_sem_map.append(ds.gt_sem.sem_map)
                gt_sem_weights.append(ds.gt_sem.sem_weights)
        return self.parse_losses(self.loss(data['inputs'], img_metas, gt_bboxes, gt_labels, 
                                           gt_bboxes_ignore = gt_bboxes_ignore, 
                                           gt_masks = gt_masks, 
                                           gt_sem_map = gt_sem_map,
                                           gt_sem_weights = gt_sem_weights))

split_rep1 = AutoSplitRepPointsV2MaskDetector.create_from_cfg_and_checkpoint('../configs/detection/mask_reppoitsv2_swin_t.py', '../checkpoints/detection/mask_repointsv2_swin_t.pth', device = device, cut_point = 1)
split_rep2 = SplitRepPointsV2MaskDetector.create_from_cfg_and_checkpoint('../configs/detection/mask_reppoitsv2_swin_t.py', '../checkpoints/detection/mask_repointsv2_swin_t.pth', device = device, cut_point = 1)

08/02 14:51:53 - mmengine - INFO - Loads checkpoint by local backend from path: ../checkpoints/detection/mask_repointsv2_swin_t.pth


/localhome/aharell/.pyenv/versions/3.10.6/lib/python3.10/subprocess.py:1070: ResourceWarning: subprocess 3072325 is still running
  _warn("subprocess %s is still running" % self.pid,


Done!!, save to ../checkpoints/detection/mask_repointsv2_swin_t_converted-d70cb43e.pth


/localhome/aharell/CFM-Task-Models/.venv/lib/python3.10/site-packages/mmdet/models/task_modules/builder.py:39: UserWarning: ``build_assigner`` would be deprecated soon, please use ``mmdet.registry.TASK_UTILS.build()`` 
  warnings.warn('``build_assigner`` would be deprecated soon, please use '
/localhome/aharell/CFM-Task-Models/.venv/lib/python3.10/site-packages/mmdet/models/task_modules/builder.py:24: UserWarning: ``build_iou_calculator`` would be deprecated soon, please use ``mmdet.registry.TASK_UTILS.build()`` 
  warnings.warn(
/localhome/aharell/CFM-Task-Models/.venv/lib/python3.10/site-packages/mmdet/models/task_modules/builder.py:46: UserWarning: ``build_sampler`` would be deprecated soon, please use ``mmdet.registry.TASK_UTILS.build()`` 
  warnings.warn('``build_sampler`` would be deprecated soon, please use '
/localhome/aharell/CFM-Task-Models/.venv/lib/python3.10/site-packages/mmdet/apis/inference.py:90: UserWarning: dataset_meta or class names are not saved in the checkpoint's

Loads checkpoint by local backend from path: ../checkpoints/detection/mask_repointsv2_swin_t_converted-d70cb43e.pth
08/02 14:51:54 - mmengine - INFO - Loads checkpoint by local backend from path: ../checkpoints/detection/mask_repointsv2_swin_t.pth


/localhome/aharell/CFM-Task-Models/.venv/lib/python3.10/site-packages/mmdet/models/task_modules/builder.py:39: UserWarning: ``build_assigner`` would be deprecated soon, please use ``mmdet.registry.TASK_UTILS.build()`` 
  warnings.warn('``build_assigner`` would be deprecated soon, please use '
/localhome/aharell/CFM-Task-Models/.venv/lib/python3.10/site-packages/mmdet/models/task_modules/builder.py:24: UserWarning: ``build_iou_calculator`` would be deprecated soon, please use ``mmdet.registry.TASK_UTILS.build()`` 
  warnings.warn(
/localhome/aharell/CFM-Task-Models/.venv/lib/python3.10/site-packages/mmdet/models/task_modules/builder.py:46: UserWarning: ``build_sampler`` would be deprecated soon, please use ``mmdet.registry.TASK_UTILS.build()`` 
  warnings.warn('``build_sampler`` would be deprecated soon, please use '
/localhome/aharell/.pyenv/versions/3.10.6/lib/python3.10/subprocess.py:1070: ResourceWarning: subprocess 3072327 is still running
  _warn("subprocess %s is still running" %

Done!!, save to ../checkpoints/detection/mask_repointsv2_swin_t_converted-d70cb43e.pth


/localhome/aharell/CFM-Task-Models/.venv/lib/python3.10/site-packages/mmdet/models/task_modules/builder.py:39: UserWarning: ``build_assigner`` would be deprecated soon, please use ``mmdet.registry.TASK_UTILS.build()`` 
  warnings.warn('``build_assigner`` would be deprecated soon, please use '
/localhome/aharell/CFM-Task-Models/.venv/lib/python3.10/site-packages/mmdet/models/task_modules/builder.py:24: UserWarning: ``build_iou_calculator`` would be deprecated soon, please use ``mmdet.registry.TASK_UTILS.build()`` 
  warnings.warn(
/localhome/aharell/CFM-Task-Models/.venv/lib/python3.10/site-packages/mmdet/models/task_modules/builder.py:46: UserWarning: ``build_sampler`` would be deprecated soon, please use ``mmdet.registry.TASK_UTILS.build()`` 
  warnings.warn('``build_sampler`` would be deprecated soon, please use '
/localhome/aharell/CFM-Task-Models/.venv/lib/python3.10/site-packages/mmdet/apis/inference.py:90: UserWarning: dataset_meta or class names are not saved in the checkpoint's

Loads checkpoint by local backend from path: ../checkpoints/detection/mask_repointsv2_swin_t_converted-d70cb43e.pth


/localhome/aharell/CFM-Task-Models/.venv/lib/python3.10/site-packages/mmdet/models/task_modules/builder.py:39: UserWarning: ``build_assigner`` would be deprecated soon, please use ``mmdet.registry.TASK_UTILS.build()`` 
  warnings.warn('``build_assigner`` would be deprecated soon, please use '
/localhome/aharell/CFM-Task-Models/.venv/lib/python3.10/site-packages/mmdet/models/task_modules/builder.py:24: UserWarning: ``build_iou_calculator`` would be deprecated soon, please use ``mmdet.registry.TASK_UTILS.build()`` 
  warnings.warn(
/localhome/aharell/CFM-Task-Models/.venv/lib/python3.10/site-packages/mmdet/models/task_modules/builder.py:46: UserWarning: ``build_sampler`` would be deprecated soon, please use ``mmdet.registry.TASK_UTILS.build()`` 
  warnings.warn('``build_sampler`` would be deprecated soon, please use '


True
